## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Port Alfred,ZA,2022-02-04 22:26:32,-33.5906,26.8910,68.54,79,100,6.64,light rain
1,1,Dali,CN,2022-02-04 22:26:32,25.7000,100.1833,40.14,80,0,2.62,clear sky
2,2,Clyde River,CA,2022-02-04 22:26:33,70.4692,-68.5914,-7.37,77,100,8.05,light snow
3,3,Selcuk,TR,2022-02-04 22:26:33,37.9514,27.3685,47.01,76,3,5.53,clear sky
4,4,Kodiak,US,2022-02-04 22:20:21,57.7900,-152.4072,36.91,75,100,4.61,overcast clouds


In [13]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [14]:
# Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Port Alfred,ZA,2022-02-04 22:26:32,-33.5906,26.8910,68.54,79,100,6.64,light rain
10,10,Bima,ID,2022-02-04 22:26:34,-8.4667,118.7167,76.59,93,91,0.49,overcast clouds
11,11,Hithadhoo,MV,2022-02-04 22:20:17,-0.6000,73.0833,81.54,74,80,11.81,broken clouds
13,13,Sharjah,AE,2022-02-04 22:26:35,25.3573,55.4033,70.25,70,0,4.61,clear sky
17,17,Mufumbwe,ZM,2022-02-04 22:26:36,-13.6833,24.8000,63.32,98,100,3.31,overcast clouds
18,18,Atuona,PF,2022-02-04 22:20:18,-9.8000,-139.0333,78.76,77,25,17.27,light rain
19,19,Caravelas,BR,2022-02-04 22:26:37,-17.7125,-39.2481,77.00,88,100,7.23,overcast clouds
20,20,Rikitea,PF,2022-02-04 22:16:13,-23.1203,-134.9692,78.44,71,12,7.72,light rain
23,23,Busselton,AU,2022-02-04 22:16:14,-33.6500,115.3333,65.08,49,0,3.53,clear sky
25,25,Carmen,MX,2022-02-04 22:26:38,18.6333,-91.8333,78.84,83,75,10.36,mist


In [15]:
# Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                353
City                   353
Country                351
Date                   353
Lat                    353
Lng                    353
Max Temp               353
Humidity               353
Cloudiness             353
Wind Speed             353
Current Description    353
dtype: int64

In [17]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

clean_df.count()

City_ID                351
City                   351
Country                351
Date                   351
Lat                    351
Lng                    351
Max Temp               351
Humidity               351
Cloudiness             351
Wind Speed             351
Current Description    351
dtype: int64

In [18]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Port Alfred,ZA,68.54,light rain,-33.5906,26.8910,
10,Bima,ID,76.59,overcast clouds,-8.4667,118.7167,
11,Hithadhoo,MV,81.54,broken clouds,-0.6000,73.0833,
13,Sharjah,AE,70.25,clear sky,25.3573,55.4033,
17,Mufumbwe,ZM,63.32,overcast clouds,-13.6833,24.8000,
18,Atuona,PF,78.76,light rain,-9.8000,-139.0333,
19,Caravelas,BR,77.00,overcast clouds,-17.7125,-39.2481,
20,Rikitea,PF,78.44,light rain,-23.1203,-134.9692,
23,Busselton,AU,65.08,clear sky,-33.6500,115.3333,
25,Carmen,MX,78.84,mist,18.6333,-91.8333,


In [19]:
# Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [54]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()


City                   351
Country                351
Max Temp               351
Current Description    351
Lat                    351
Lng                    351
Hotel Name             351
dtype: int64

In [29]:
# Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [39]:
# Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [40]:
# Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Display the figure
fig

Figure(layout=FigureLayout(height='420px'))